# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import classification_report , confusion_matrix

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

# Load and Prepare Dataset

In [13]:
X_train = []
y_train = []
labels = ["cardboard" , "glass" , "metal" , "paper" , "plastic" , "trash"]
image_size = 224

for label in labels :
    folderPath = os.path.join("/Users/donaldheddesheimer/Documents/GitHub/Smart-Bin/src/Garbage-classification" , label)
    for path in os.listdir(folderPath) :
        image_path = os.path.join(folderPath , path)
        image = cv2.imread(image_path)
        if image is not None:  # Check if the image was successfully read
            image = cv2.resize(image , (image_size , image_size))
            X_train.append(image)
            y_train.append(label)


In [14]:
X_test = []
y_test = []

for label in labels :
    folderPath = os.path.join("/Users/donaldheddesheimer/Documents/GitHub/Smart-Bin/src/Garbage-classification" , label)
    for path in os.listdir(folderPath) :
        image_path = os.path.join(folderPath , path)
        image = cv2.imread(image_path)
        image = cv2.resize(image , (image_size , image_size))
        X_test.append(image)
        y_test.append(label)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/garbage-classification/Garbage classification/Garbage classification/cardboard'

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
X_train , y_train = shuffle(X_train , y_train , random_state = 42)

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
y_train = to_categorical(y_train , num_classes = 6)
y_test = to_categorical(y_test , num_classes = 6)

In [ ]:
print(f" X_train : {X_train.shape} ")
print(f" X_test : {X_test.shape} ")
print(f" y_train : {y_train.shape} ")
print(f" y_test : {y_test.shape} ")

In [ ]:
datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

In [ ]:
datagen.fit(X_train)

# ResNet50 Model

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
output = Dense(6, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)

In [ ]:
history = model.fit(X_train , y_train , epochs = 60 , batch_size = 64 , verbose = 1 ,
                   validation_data = (X_test , y_test) , callbacks = [early_stopping, reduce_lr])

# Evaluation

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred , axis = 1)
y_test = np.argmax(y_test , axis = 1)

In [ ]:
print(classification_report(y_pred , y_test , target_names = labels))

In [ ]:
plt.figure(figsize = (8,6))
sns.lineplot(history.history["accuracy"] , label = "Train Accuracy")
sns.lineplot(history.history["val_accuracy"] , label = "Test Accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("Train vs Test Accuracy ")
plt.legend()
plt.show()

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
model.save('/kaggle/working/resnet_model.h5')